## Setup

In [48]:
from functions import Model, get99, splitData, dropMissingValues, z_score, outliers_knn, outliers_dbscan, getNoise, getRelFeatures, drop_features, getCombinations, get_unique_list, bold, blue, red, green, getBestModel, np_to_df, df_to_np, drop99_all, z_score_individual, train_test_to_df, saveModel, loadModel
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
import json
import numpy as np
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from functions import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

Json Hyper Parameters

In [49]:
# JSON file
f = open ('data/json/input.json', "r")
  # Reading from file
PARAMETERS_ALL = json.loads(f.read())
PARAMETERS = PARAMETERS_ALL["GRID_SEARCH"]
SPLIT_SIZE = PARAMETERS["SPLIT_SIZE"]

OUTLIER99 = PARAMETERS["OUTLIER99"] 
OUTLIER_ZSCORE = {"STD" : PARAMETERS["OUTLIER_ZSCORE"]["STD"]}
OUTLIER_ZSCORE_IND = PARAMETERS["OUTLIER_ZSCORE_IND"]
OUTLIER_KNN = {"K": PARAMETERS["OUTLIER_KNN"]["K"], "X": PARAMETERS["OUTLIER_KNN"]["X"]}
OUTLIER_DBSCAN = {"K": PARAMETERS["OUTLIER_DBSCAN"]["K"], "X": PARAMETERS["OUTLIER_DBSCAN"]["X"], "EPS": PARAMETERS["OUTLIER_DBSCAN"]["EPS"], "MIN_SAMPLES":  PARAMETERS["OUTLIER_DBSCAN"]["MIN_SAMPLES"]}
OUTLIER_NOISE = {"CV": PARAMETERS["OUTLIER_NOISE"]["CV"], "DEPTH": PARAMETERS["OUTLIER_NOISE"]["DEPTH"], "MULTI": PARAMETERS["OUTLIER_NOISE"]["MULTI"] }

FEATURE_SEL_CORR = {"THRESHOLD_CORR": PARAMETERS["FEATURE_SEL_CORR"]["THRESHOLD"]}
FEATURE_SEL_MANUAL = {"FEATURE_SEL_MANUAL": PARAMETERS["FEATURE_SEL_MANUAL"]}

REG_LINEAR = PARAMETERS["REG_LINEAR"]
REG_LASSO = {"ALPHA": PARAMETERS["REG_LASSO"]["ALPHA"]}

f.close()

#TODO FEATURE
#TODO Regressionmodels
#TODO TESTDATA --> path to csv-file


Get Data from CSV

In [50]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])

maeList = []#for model selection

Missing Values

In [51]:
df = dropMissingValues(df) 

if OUTLIER99 == True:
    print("outlier_99")
    outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
    df = drop99_all(df, outlier_list_99 )
    print(green('done'), "\n")

outlier_99
done 



## Outlier Detection

In [53]:
outlier_dict_all = {}

print('starting z_score')
for p in OUTLIER_ZSCORE["STD"]:
    print(p)
    outlier_list_z_score = z_score(df, p)
    outlier_dict_all["z_score_"+str(p)] = outlier_list_z_score

if OUTLIER_ZSCORE_IND == True:
    outlier_list_z_score_ind = z_score_individual(df)

print(green('done'), '\n\nstarting knn')
for k in OUTLIER_KNN["K"]:
    for o in OUTLIER_KNN["X"]:
        print(k, o)
        outlier_list_knn = outliers_knn(df, k, o, SPLIT_SIZE)
        outlier_dict_all["knn_"+str(k)+"_"+str(o)] = outlier_list_knn

print(green('done'), '\n\nstarting db_scan')
for k in OUTLIER_DBSCAN["K"]:
    for o in OUTLIER_DBSCAN["X"]:
        for e in OUTLIER_DBSCAN["EPS"]:
            for m in OUTLIER_DBSCAN["MIN_SAMPLES"]:
                print(k, o, e, m)
                outlier_list_dbscan = outliers_dbscan(df, k, o, e, m, SPLIT_SIZE)
                outlier_dict_all["db_scan_"+str(k)+"_"+str(o)+"_"+str(e)+"_"+str(m)] = outlier_list_dbscan

print(green('done'), '\n\nstarting noise')
for cv in OUTLIER_NOISE["CV"]:
    for d in OUTLIER_NOISE["DEPTH"]:
        for m in OUTLIER_NOISE["MULTI"]:
            print(cv, d, m)
            outlier_list_noise = getNoise(df, cv, d, m)
            outlier_dict_all["noise_"+str(cv)+"_"+str(d)+"_"+str(m)] = outlier_list_noise

print(green('done'))

starting z_score
10.1
done 

starting knn
3 1500
done 

starting db_scan
4 500 0.6 8
4 500 0.4 8
done 

starting noise
5 5 10
done


## Combinations Outlier / Noise 

In [54]:
outlier_lists_all = list(outlier_dict_all.values())
outlier_lists_keys = list(outlier_dict_all.keys())
combination_list = []
for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
    if not i == 1: #emtpy tuple
        combination_list.append(combo)
        # print('combo #{}: {}'.format(i, combo))
print("...")
for c in combination_list[10:]:
    print(bold("Combination: "), blue(c),"\n",)
print(bold(len(combination_list)), " total Combinations")

...
Combination:  ('knn_3_1500', 'db_scan_4_500_0.4_8') 

Combination:  ('knn_3_1500', 'noise_5_5_10') 

Combination:  ('db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8') 

Combination:  ('db_scan_4_500_0.6_8', 'noise_5_5_10') 

Combination:  ('db_scan_4_500_0.4_8', 'noise_5_5_10') 

Combination:  ('z_score_10.1', 'knn_3_1500', 'db_scan_4_500_0.6_8') 

Combination:  ('z_score_10.1', 'knn_3_1500', 'db_scan_4_500_0.4_8') 

Combination:  ('z_score_10.1', 'knn_3_1500', 'noise_5_5_10') 

Combination:  ('z_score_10.1', 'db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8') 

Combination:  ('z_score_10.1', 'db_scan_4_500_0.6_8', 'noise_5_5_10') 

Combination:  ('z_score_10.1', 'db_scan_4_500_0.4_8', 'noise_5_5_10') 

Combination:  ('knn_3_1500', 'db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8') 

Combination:  ('knn_3_1500', 'db_scan_4_500_0.6_8', 'noise_5_5_10') 

Combination:  ('knn_3_1500', 'db_scan_4_500_0.4_8', 'noise_5_5_10') 

Combination:  ('db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8', 'noise_5_5_10') 

## Algorythm Selection

### Linear Regression

In [55]:
model_obj_list = []

In [56]:
if REG_LINEAR == True:

    for i, c in enumerate(combination_list):
        print(f'\n{bold(i)}: {blue(c)}')
        outlier_list = get_unique_list(outlier_dict_all, c)
        try: 
            X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop") #TODO drop entfernen
            model = LinearRegression()
            obj = Model(model, (X_train, X_test, y_train, y_test), c, df, df.columns.to_list())
            model_obj_list.append(obj)

        except Exception as e:
            print(bold("Error"), red(e))


0: ('z_score_10.1',)
dropped 41 / 53 rows

1: ('knn_3_1500',)
dropped 1500 / 1500 rows

2: ('db_scan_4_500_0.6_8',)
dropped 525 / 525 rows

3: ('db_scan_4_500_0.4_8',)
dropped 3783 / 3783 rows

4: ('noise_5_5_10',)
dropped 35 / 51 rows

5: ('z_score_10.1', 'knn_3_1500')
dropped 1536 / 1536 rows

6: ('z_score_10.1', 'db_scan_4_500_0.6_8')
dropped 562 / 562 rows

7: ('z_score_10.1', 'db_scan_4_500_0.4_8')
dropped 3814 / 3814 rows

8: ('z_score_10.1', 'noise_5_5_10')
dropped 76 / 76 rows

9: ('knn_3_1500', 'db_scan_4_500_0.6_8')
dropped 1534 / 1534 rows

10: ('knn_3_1500', 'db_scan_4_500_0.4_8')
dropped 3783 / 3783 rows

11: ('knn_3_1500', 'noise_5_5_10')
dropped 1532 / 1532 rows

12: ('db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8')
dropped 3783 / 3783 rows

13: ('db_scan_4_500_0.6_8', 'noise_5_5_10')
dropped 559 / 559 rows

14: ('db_scan_4_500_0.4_8', 'noise_5_5_10')
dropped 3808 / 3808 rows

15: ('z_score_10.1', 'knn_3_1500', 'db_scan_4_500_0.6_8')
dropped 1569 / 1569 rows

16: ('z_score

### Lasso Regression

In [64]:
#model_obj_list = []
for i, c in enumerate(combination_list):
    print(f'\n{bold(i)}: {blue(c)}')
    outlier_list = get_unique_list(outlier_dict_all, c)
    try: 
        X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")
        df_lasso = train_test_to_df(X_train, X_test, y_train, y_test, df.columns.to_list())
        X_l, y_l = df_lasso.drop(columns=["price", "date", "id"]), df_lasso["price"]
        scaler = StandardScaler().fit(X_l)
        X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_l, y_l, test_size = SPLIT_SIZE, random_state=1) #TODO 
        X_train_l = scaler.transform(X_train_l)
        X_test_l = scaler.transform(X_test_l)

        for a in REG_LASSO["ALPHA"]:
            reg = Lasso(alpha=a)
            obj_l = Model(reg, (X_train_l, X_test_l, y_train_l, y_test_l), c, df_lasso, X_l.columns.to_list())
            model_obj_list.append(obj_l)

    except Exception as e:
        print(bold("Error"), red(e))


0: ('z_score_10.1',)
dropped 41 / 41 rows

1: ('knn_3_1500',)
dropped 1500 / 1500 rows

2: ('db_scan_4_500_0.6_8',)
dropped 525 / 525 rows

3: ('db_scan_4_500_0.4_8',)
dropped 3783 / 3783 rows

4: ('noise_5_5_10',)
dropped 35 / 35 rows

5: ('z_score_10.1', 'knn_3_1500')
dropped 1536 / 1536 rows

6: ('z_score_10.1', 'db_scan_4_500_0.6_8')
dropped 562 / 562 rows

7: ('z_score_10.1', 'db_scan_4_500_0.4_8')
dropped 3814 / 3814 rows

8: ('z_score_10.1', 'noise_5_5_10')
dropped 76 / 76 rows

9: ('knn_3_1500', 'db_scan_4_500_0.6_8')
dropped 1534 / 1534 rows

10: ('knn_3_1500', 'db_scan_4_500_0.4_8')
dropped 3783 / 3783 rows

11: ('knn_3_1500', 'noise_5_5_10')
dropped 1532 / 1532 rows

12: ('db_scan_4_500_0.6_8', 'db_scan_4_500_0.4_8')
dropped 3783 / 3783 rows

13: ('db_scan_4_500_0.6_8', 'noise_5_5_10')
dropped 559 / 559 rows

14: ('db_scan_4_500_0.4_8', 'noise_5_5_10')
dropped 3808 / 3808 rows

15: ('z_score_10.1', 'knn_3_1500', 'db_scan_4_500_0.6_8')
dropped 1569 / 1569 rows

16: ('z_score

In [65]:
#Summary
df_summary = pd.DataFrame(columns=['model', 'combo', 'mae', 'score'])
for o in model_obj_list:
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    row = {'model': o.get_model(), 'combo': comb, 'mae': mae, 'score': score}
    df_summary = df_summary.append(row, ignore_index=True)

df_summary.sort_values("mae")

,model,combo,mae,score
168,Lasso(alpha=5),"(db_scan_4_500_0.4_8, noise_5_5_10)",100319.23,0.677605
213,Lasso(alpha=5),"(knn_3_1500, db_scan_4_500_0.6_8, db_scan_4_50...",100319.23,0.677605
195,Lasso(alpha=5),"(knn_3_1500, db_scan_4_500_0.4_8, noise_5_5_10)",100319.23,0.677605
198,Lasso(alpha=5),"(db_scan_4_500_0.6_8, db_scan_4_500_0.4_8, noi...",100319.23,0.677605
212,Lasso(alpha=2),"(knn_3_1500, db_scan_4_500_0.6_8, db_scan_4_50...",100320.24,0.677608
...,...,...,...,...
10,LinearRegression(),"(knn_3_1500, db_scan_4_500_0.4_8)",162867.97,0.431331
12,LinearRegression(),"(db_scan_4_500_0.6_8, db_scan_4_500_0.4_8)",162867.97,0.431331
0,LinearRegression(),"(z_score_10.1,)",165149.5,0.525593
8,LinearRegression(),"(z_score_10.1, noise_5_5_10)",165189.16,0.525594


In [66]:
mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
#score_best = df_summary[df_summary.score==df_summary.score.max()]
best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
print(best_model_obj)
#rint(f'{bold("Lowest mae:")} {mae_best}')

Model: Lasso(alpha=5) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 22


In [67]:
X_train, X_test, y_train, y_test = best_model_obj.get_data()

df_test =  train_test_to_df(X_train, X_test, y_train, y_test,best_model_obj.get_features())

df_test.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,dis_super,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,ahf1,ahf2,ahf3,price
0,0.846881,2.045170,1.362121,-0.194248,0.926698,-0.056309,0.878616,-0.181613,-0.604372,0.406741,...,-0.119316,1.459420,-0.832892,-0.875961,1.539933,-0.168457,-0.389593,0.523668,-0.079872,600600.0
1,-0.357813,-0.419883,-0.632031,-0.042883,0.926698,-0.056309,-0.556745,-0.181613,-0.604372,-0.530558,...,-0.119316,-1.202982,1.501553,-0.052853,-0.044456,-0.000974,0.246870,-0.981943,-0.272403,375000.0
2,0.846881,0.636568,0.186996,-0.197279,0.926698,-0.056309,1.380290,-0.181613,-0.604372,-0.530558,...,-0.119316,-0.833744,-1.339310,0.201563,0.300659,-0.195476,-0.307509,0.825083,0.141536,275000.0
3,0.846881,-0.067733,-0.430242,-0.192955,-0.915226,-0.056309,-1.047273,-0.181613,-0.604372,-0.530558,...,-0.119316,-1.358451,-1.992026,-0.763719,-0.091517,-0.166796,-1.222073,-0.531708,-1.330880,280000.0
4,0.846881,0.636568,1.302772,-0.268172,0.926698,-0.056309,-0.742140,-0.181613,-0.604372,1.344040,...,-0.119316,-0.270170,-0.097180,0.171631,1.477185,-0.272380,-1.088810,-0.516972,-1.207239,710000.0


## Feature Selection

In [68]:
model_obj_list_best = [best_model_obj]
X_train, X_test, y_train, y_test = best_model_obj.get_data()

df_feature_reduction = train_test_to_df(X_train, X_test, y_train, y_test, best_model_obj.get_features())
#print(df_feature_reduction)

for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
    #print(threshold, type(threshold))
    try: 
        rel_features = getRelFeatures(df_feature_reduction, threshold)
        #print(len(rel_features), rel_features)
        df_feature_reduction = df_feature_reduction[rel_features]
        X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
        best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction, df_feature_reduction.columns.to_list())
        model_obj_list_best.append(best_model_obj_new)

    except Exception as e:
        print(bold("Error"), red(e))

for m in model_obj_list_best:
    print(m)

Model: Lasso(alpha=5) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 23
Model: Lasso(alpha=5) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 10
Model: Lasso(alpha=5) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 9


In [69]:
model_obj_list_best = [best_model_obj]
X_train, X_test, y_train, y_test = best_model_obj.get_data()

df_feature_reduction_manual = train_test_to_df(X_train, X_test, y_train, y_test, best_model_obj.get_features())
#print(df_feature_reduction)

rel_features = [
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'dis_super',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15',
 'ahf1',
 'ahf2',
 'ahf3']

 # 'yr_renovated',
 #'zipcode',

#print(len(rel_features), rel_features)
df_feature_reduction_manual = df_feature_reduction_manual[rel_features]
X_train, X_test, y_train, y_test = splitData(df_feature_reduction_manual, SPLIT_SIZE)
best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction_manual, df_feature_reduction_manual.columns.to_list())
model_obj_list_best.append(best_model_obj_new)

for m in model_obj_list_best:
    print(m)

Model: Lasso(alpha=5) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 23
Model: Lasso(alpha=5) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 21


In [70]:
#Summary
df_summary_best = pd.DataFrame(columns=['model', 'combo', 'mae', 'score', 'features'])
for i, o in enumerate(model_obj_list_best):
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    features = o.get_features()
    if "price" in features:
        features.remove("price")
    row = {'model': o.get_model(),'combo': comb, 'mae': mae, 'score': score, 'features': len(features), 'index':int(i)}
    df_summary_best = df_summary_best.append(row, ignore_index=True)
    i +=1

df_summary_best.sort_values("mae")

,model,combo,mae,score,features,index
0,Lasso(alpha=5),"(db_scan_4_500_0.4_8, noise_5_5_10)",100319.23,0.677605,22,0.0
1,Lasso(alpha=5),"(db_scan_4_500_0.4_8, noise_5_5_10)",102109.12,0.688054,20,1.0


## Select Best Model

In [71]:
saveModel(model_obj_list_best[0])
loaded_object = loadModel()

Model: Lasso(alpha=5) Combination: ('db_scan_4_500_0.4_8', 'noise_5_5_10') Features: 22
